In [1]:
import os
import glob
import json
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from typing import List, Tuple

## Funções Auxiliares (Carregamento, Salvamento e Visualização)

In [3]:
def desserializar_keypoints(arr: np.ndarray) -> List[cv2.KeyPoint]:
    """
    Reconstrói lista de cv2.KeyPoint a partir do array (N,7).
    """
    kps = []
    for row in arr:
        k = cv2.KeyPoint(
            x=float(row[0]), y=float(row[1]), size=float(row[2]),
            angle=float(row[3]), response=float(row[4]),
            octave=int(row[5]), class_id=int(row[6])
        )
        kps.append(k)
    return kps


def carregar_features_npz(caminho_npz: str) -> Tuple[List[cv2.KeyPoint], np.ndarray, str]:
    """
    Lê um .npz salvo pela Etapa 2 e retorna (keypoints, descriptors, imagem_absoluta).
    """
    with np.load(caminho_npz, allow_pickle=True) as data:
        kps = desserializar_keypoints(data["keypoints"])
        desc = data["descriptors"]
        img_path = str(data["imagem_absoluta"])
    return kps, desc, img_path


def carregar_matches_npz(caminho_npz) -> List[cv2.DMatch]:
    """
    Carrega os matches salvos em .npz e restaura cada linha como um objeto cv2.DMatch.
    Retorna uma lista de cv2.DMatch.
    """

    with np.load(caminho_npz, allow_pickle=True) as data:
        arr = data["matches"]  # array shape (N, 3)
        matches = []
        for row in arr:
            # cv2.DMatch(queryIdx, trainIdx, distance)
            m = cv2.DMatch(_queryIdx=int(row[0]), _trainIdx=int(row[1]), _distance=float(row[2]))
            matches.append(m)
    return matches


def calcular_homogr(kps1, kps2, matches, params = None, direction = '2to1'):
    """
    Calcula a homografia entre duas imagens usando os pontos correspondentes.
    """
    if not kps1 or not kps2 or not matches or len(matches) < 4:
        return None, None

    # Extrai os pontos-chave e descritores
    pts1 = [kps1[m.queryIdx].pt for m in matches]
    pts2 = [kps2[m.trainIdx].pt for m in matches]

    src, dst = (pts2, pts1) if direction == "2to1" else (pts1, pts2)

    # Calcula a homografia usando RANSAC
    H, mask = cv2.findHomography(np
                                 .array(src, dtype=np.float32), np.array(dst, dtype=np.float32), cv2.RANSAC, **(params or {}))

    return H, mask

def alinhar_imgs(img1, img2, homografia):
    if homografia is None:
        return None
    
    h1, w1 = img1.shape[:2]
    h2, w2 = img2.shape[:2]

    # Cantos da imagem 2
    cantos2 = np.array([[0,0],[w2,0],[w2,h2],[0,h2]], dtype=np.float32).reshape(-1,1,2)

    # Aplica a homografia nos cantos da imagem 2
    cantos2_trans = cv2.perspectiveTransform(cantos2, homografia)

    # Combina com os cantos da imagem 1 para calcular o retangulo final
    cantos_totais = np.vstack((cantos2_trans, np.array([[0,0],[w1,0],[w1,h1],[0,h1]], dtype=np.float32).reshape(-1,1,2)))
    xmin, ymin = np.floor(cantos_totais.min(axis=0).ravel()).astype(np.int32)
    xmax, ymax = np.ceil(cantos_totais.max(axis=0).ravel()).astype(np.int32)

    # Translacao para coordenadas positivas
    translacao = [-xmin, -ymin]
    T = np.array([[1,0,translacao[0]], [0,1,translacao[1]], [0,0,1]], dtype=np.float64)

    # Aplica warpPerspective com a homografia ajustada
    res = cv2.warpPerspective(img2, T @ homografia, (xmax - xmin, ymax - ymin))

    # Coloca img1 na posição correta
    res[translacao[1]:translacao[1]+h1, translacao[0]:translacao[0]+w1] = img1

    return res

def salvar_homogr_npz(dir_saida, i, j, H, mask, path1, path2, metrics, params=None, direction="2to1"):
    os.makedirs(dir_saida, exist_ok=True)
    mask_arr = np.asarray(mask, dtype=np.uint8).reshape(-1,1) if mask is not None else np.zeros((0,1), np.uint8)
    np.savez_compressed(
        os.path.join(dir_saida, f"homogr_{i:03d}_{j:03d}.npz"),
        H=np.asarray(H, dtype=np.float32) if H is not None else np.zeros((3,3), np.float32),
        mask=mask_arr,
        inliers=np.int32(metrics.get("inliers", 0)),
        total=np.int32(metrics.get("total", 0)),
        rmse_px=np.float32(metrics.get("rmse_px") if metrics.get("rmse_px") is not None else np.nan),
        direction=str(direction),                 # "2to1" => img2 -> img1
        params_json=json.dumps(params or {}),     # p/ reprodutibilidade
        idx_i=np.int32(i), idx_j=np.int32(j),
        path_i=str(path1), path_j=str(path2)
    )

def carregar_homogr_npz(caminho_npz):
    """
    Lê homogr_*.npz e retorna:
      H (3x3 float32), mask (Nx1 uint8), meta (dict: inliers, total, rmse_px, direction, params_json, idx_i, idx_j, path_i, path_j)
    """
    with np.load(caminho_npz, allow_pickle=True) as d:
        H = d["H"].astype(np.float32)
        mask = d["mask"].astype(np.uint8)
        meta = {
            "inliers": int(d["inliers"]),
            "total": int(d["total"]),
            "rmse_px": None if np.isnan(d["rmse_px"]) else float(d["rmse_px"]),
            "direction": str(d["direction"]),
            "params_json": str(d["params_json"]),
            "idx_i": int(d["idx_i"]), "idx_j": int(d["idx_j"]),
            "path_i": str(d["path_i"]), "path_j": str(d["path_j"]),
        }
    return H, mask, meta

def carregar_homogr_dir(dir_homogr):
    arquivos = sorted(glob.glob(os.path.join(dir_homogr, "homogr_*.npz")))
    dados = []
    for f in arquivos:
        H, mask, meta = carregar_homogr_npz(f)
        dados.append((H, mask, meta, f))
    return dados  # lista de tuplas (H, mask, meta, caminho)

def _rmse_reproj(H, pts_src, pts_dst, mask):
    if H is None or mask is None or not mask.any():
        return None
    sel = mask.ravel().astype(bool)
    if sel.sum() < 4:
        return None
    src = np.float32(pts_src)[sel]  # pts_src no sistema da imagem de origem em H
    dst = np.float32(pts_dst)[sel]  # pts_dst no sistema de destino
    src_h = cv2.convertPointsToHomogeneous(src).reshape(-1,3).T
    proj = (H @ src_h); proj = (proj[:2] / proj[2]).T
    dif = dst - proj
    return float(np.sqrt((dif**2).sum(axis=1).mean()))

def listar_pares_de_matches(caminho_matcher_dir, only_sequential: bool = True, n_imgs: int | None = None):
     """
     Lê os arquivos matches_*.npz e retorna:
       - only_sequential=True: apenas pares sequenciais com wrap (i, (i+1)%n)
       - only_sequential=False: todos os pares encontrados
     Se n_imgs não for passado, tenta inferir n a partir dos índices observados.
     """
     arquivos = sorted(glob.glob(os.path.join(caminho_matcher_dir, "matches_*.npz")))
     parsed = []
     for arq in arquivos:
         base = os.path.splitext(os.path.basename(arq))[0]  # matches_000_001
         _, si, sj = base.split("_")
         parsed.append((int(si), int(sj), arq))
 
     if not only_sequential:
         return parsed
 
     # Filtra para pares sequenciais com wrap
     if n_imgs is None:
         idxs = set()
         for i, j, _ in parsed:
             idxs.add(i); idxs.add(j)
         n = max(idxs) + 1 if idxs else 0
     else:
         n = int(n_imgs)
 
     seq = {(i, (i + 1) % n) for i in range(n)}
     filtered = [t for t in parsed if (t[0], t[1]) in seq]
     # Ordenação estável por (i, j) ajuda na leitura/depuração
     filtered.sort(key=lambda t: (t[0], t[1]))
     return filtered



## Pipeline Principal da Etapa 4

In [24]:
def analisar_homogr_alinhamento(caminho_base_etapa2: str, caminho_base_etapa3: str, detectores: List[str], matchers: List[str], candidatos: List[dict] | None = None, sequential_pairs: bool = True):
    """
    Pipeline da Etapa 4: Carrega features e matches, calcula homografia com RANSAC e alinha imagens com warpPerspective.
    """
    print(f"\n{'='*65}\nHomografia e alinhamento para o conjunto: '{os.path.basename(caminho_base_etapa3)}'\n{'='*65}")

    # Carrega a ordem das imagens
    caminho_json = os.path.join(caminho_base_etapa2, "ordem_imagens.json")
    try:
        with open(caminho_json, "r") as f:
            arquivos_de_imagem_abs = json.load(f)
    except FileNotFoundError:
        print(f"[Erro] 'ordem_imagens.json' não encontrado em '{caminho_base_etapa2}'. Abortando.")
        return
    
    # Itera sobre os detectores (sift, orb, etc.)
    for nome_detector in detectores:
        print(f"\n--- Detector: {nome_detector.upper()} ---")
        caminho_detector = os.path.join(caminho_base_etapa2, nome_detector)
        arquivos_feature_npz = sorted(glob.glob(os.path.join(caminho_detector, "*_features.npz")))
        if len(arquivos_feature_npz) < 2:
            continue

        # Itera sobre os matchers (bf, flann)
        for nome_matcher in matchers:
            print(f"  - Matcher: {nome_matcher.upper()}")
            caminho_matcher = os.path.join(caminho_base_etapa3, nome_detector, nome_matcher)

            # Cria o diretório de saída para esta combinação
            dir_saida = os.path.join("resultados_etapa4", os.path.basename(caminho_base_etapa3), nome_detector, nome_matcher)
            os.makedirs(dir_saida, exist_ok=True)
            # NEW: processa todos os pares disponíveis na Etapa 3 (sequenciais ou todas combinações)
            pares = listar_pares_de_matches(caminho_matcher, only_sequential=sequential_pairs)
            if not pares:
                print("    [Aviso] Nenhum matches_*.npz encontrado.")
                continue

            for (i, j, caminho_npz_matches) in pares:
                # Carrega features/paths das imagens i, j
                kps1, desc1, path1 = carregar_features_npz(arquivos_feature_npz[i])
                kps2, desc2, path2 = carregar_features_npz(arquivos_feature_npz[j])
                img1 = cv2.imread(path1); img2 = cv2.imread(path2)

                # Carrega matches do par
                match_data = carregar_matches_npz(caminho_npz_matches)
                if not kps1 or not kps2 or not match_data:
                    salvar_homogr_npz(dir_saida, i, j, None, None, path1, path2,
                                      metrics={"inliers":0,"total":0,"rmse_px":None},
                                      params=None, direction="2to1")
                    continue

                # Calcula H_21 (img2 -> img1) e mask (reaproveita sua calcular_homogr)
                ransac_params = {}
                for p in candidatos or [{}]:
                    homografia, mask = calcular_homogr(kps1, kps2, match_data, p, '2to1')
                    if homografia is not None and mask is not None and int(mask.sum()) >= 4:
                        ransac_params = p
                        break # Encontrou uma homografia

                # Métricas úteis
                pts1 = [kps1[m.queryIdx].pt for m in match_data]  # destino (img1)
                pts2 = [kps2[m.trainIdx].pt for m in match_data]  # origem  (img2)
                inliers = int(mask.sum()) if (mask is not None) else 0
                total   = int(len(match_data))
                rmse    = _rmse_reproj(homografia, pts2, pts1, mask)

                # Salva para uso posterior (Etapa 5)
                salvar_homogr_npz(dir_saida, i, j, homografia, mask, path1, path2,
                                  metrics={"inliers":inliers,"total":total,"rmse_px":rmse},
                                  params=ransac_params, direction="2to1")

                # (Opcional) Preview rápido para alguns pares
                if (i, j) in [(0, 1), (1, 2)] and homografia is not None and img1 is not None and img2 is not None:
                    alinhado = alinhar_imgs(img1, img2, homografia)
                    if alinhado is not None:
                        plt.figure(figsize=(20, 10))
                        plt.imshow(cv2.cvtColor(alinhado, cv2.COLOR_BGR2RGB))
                        plt.axis('off'); plt.show()


## Execução do Pipeline para os Conjuntos de Imagens

In [ ]:
RAIZ_ETAPA2 = "resultados_etapa2"
RAIZ_ETAPA3 = "resultados_etapa3"
DETECTORS = ['sift', 'orb', 'akaze']
MATCHERS = ['bf', 'flann']

conjuntos = [d for d in os.listdir(RAIZ_ETAPA2) if os.path.isdir(os.path.join(RAIZ_ETAPA2, d))]

candidatos = [ #Parâmetros do RANSAC em grau decrescente de rigor. Se o atual não gerar um homografia válida, tenta o próximo. Tudo fica salvo no .npz
    {"ransacReprojThreshold": 0.75, "confidence": 0.9999},
    {"ransacReprojThreshold": 1.0, "confidence": 0.9997},
    {"ransacReprojThreshold": 1.5, "confidence": 0.9995},
    {}
]

for nome_conjunto in sorted(conjuntos):
    caminho_detector = os.path.join(RAIZ_ETAPA2, nome_conjunto)
    caminho_matcher = os.path.join(RAIZ_ETAPA3, nome_conjunto)
    analisar_homogr_alinhamento(caminho_detector, caminho_matcher, DETECTORS, MATCHERS, candidatos, sequential_pairs=False) #ATENÇÃO, sequential_pairs=False APENAS PARA CALCULO DO GRAFO

## Etapa bônus: identificação da ordem das imagens com inliers

In [9]:
import networkx as nx
import glob
import numpy as np
import os
import random

def ciclo_guloso(G, inicio, mais_leve):
    """
    Encontra um ciclo começando e terminando em 'inicio', sempre escolhendo o vizinho de menor (mais_leve=True)
    ou maior (mais_leve=False) peso, exceto o pai e já visitados, até retornar ao início ou não haver mais opções.
    """
    caminho = [inicio]
    atual = inicio
    pai = None
    visitados = set([inicio])
    while True:
        vizinhos = [v for v in G.neighbors(atual) if v != pai and v not in visitados]
        # Permite voltar ao início se todos os outros já foram visitados
        if not vizinhos and inicio in G.neighbors(atual):
            caminho.append(inicio)
            print("Ciclo completo")
            break
        if not vizinhos:
            print("Sem mais vizinhos disponíveis, ciclo incompleto")
            break
        if mais_leve:
            proximo = min(vizinhos, key=lambda v: G[atual][v]['weight'])
        else:
            proximo = max(vizinhos, key=lambda v: G[atual][v]['weight'])
        caminho.append(proximo)
        visitados.add(proximo)
        pai, atual = atual, proximo
    return caminho

def construir_grafo_de_homografias(caminho_base_etapa4: str, conjunto_imagens: str, detector: str, matcher: str, peso='inliers'):
    """
    Constrói um grafo direcionado onde cada nó é uma imagem e cada aresta é uma homografia entre imagens.
    O peso da aresta pode ser 'inliers' (número de inliers, maior é melhor) ou 'rmse' (menor é melhor).
    """
    dir_homogr = os.path.join(caminho_base_etapa4, conjunto_imagens, detector, matcher)
    arquivos_homogr = sorted(glob.glob(os.path.join(dir_homogr, "homogr_*.npz")))
    G = nx.Graph()
    
    for arq in arquivos_homogr:
        _, _, meta = carregar_homogr_npz(arq)
        i, j = meta['idx_i'], meta['idx_j']
        path_i, path_j = meta['path_i'], meta['path_j']
        inliers = meta['inliers']
        rmse = meta['rmse_px']
        
        if peso == 'inliers':
            if inliers > 0:
                G.add_edge(f"{path_i}", f"{path_j}", weight=inliers)
        elif peso == 'rmse':
            if rmse is not None and rmse > 0:
                G.add_edge(f"{path_i}", f"{path_j}", weight=rmse)
        else:
            raise ValueError("Peso deve ser 'inliers' ou 'rmse'.")
    
    return G

### Usando número de inliers

In [8]:
# Exemplo de uso com inliers como peso
caminho_base_etapa4 = "resultados_etapa4"
conjunto_imagens = "images-developed-png-8bit"
detector = "sift"
matcher = "flann"
peso = "inliers"

G = construir_grafo_de_homografias(caminho_base_etapa4, conjunto_imagens, detector, matcher, peso)

# inicio = random.choice(list(G.nodes()))   # Escolha um nó aleatório como início
inicio = list(G.nodes())[0]                 # Ou escolha um nó específico conhecido

ciclo = ciclo_guloso(G, inicio, mais_leve=False)
print(f"Ciclo guloso começando e terminando em '{inicio}':", ciclo)

# Salva o ciclo em um arquivo json
with open(os.path.join(caminho_base_etapa4, conjunto_imagens, detector, matcher, "ordem.json"), "w") as f:
    json.dump(ciclo, f)

Ciclo completo
Ciclo guloso começando e terminando em 'images-developed-png-8bit/panorama360raw-1.png': ['images-developed-png-8bit/panorama360raw-1.png', 'images-developed-png-8bit/panorama360raw-2.png', 'images-developed-png-8bit/panorama360raw-3.png', 'images-developed-png-8bit/panorama360raw-4.png', 'images-developed-png-8bit/panorama360raw-5.png', 'images-developed-png-8bit/panorama360raw-6.png', 'images-developed-png-8bit/panorama360raw-7.png', 'images-developed-png-8bit/panorama360raw-8.png', 'images-developed-png-8bit/panorama360raw-9.png', 'images-developed-png-8bit/panorama360raw-10.png', 'images-developed-png-8bit/panorama360raw-11.png', 'images-developed-png-8bit/panorama360raw-12.png', 'images-developed-png-8bit/panorama360raw-13.png', 'images-developed-png-8bit/panorama360raw-14.png', 'images-developed-png-8bit/panorama360raw-15.png', 'images-developed-png-8bit/panorama360raw-16.png', 'images-developed-png-8bit/panorama360raw-17.png', 'images-developed-png-8bit/panorama36

### Usando rmse (não está funcionando bem)

In [ ]:
# Exemplo de uso com rmse como peso
caminho_base_etapa4 = "resultados_etapa4"
conjunto_imagens = "images-developed-png-8bit"
detector = "sift"
matcher = "flann"
peso = "rmse"

G = construir_grafo_de_homografias(caminho_base_etapa4, conjunto_imagens, detector, matcher, peso)

# inicio = random.choice(list(G.nodes()))   # Escolha um nó aleatório como início
inicio = list(G.nodes())[0]                 # Ou escolha um nó específico conhecido

ciclo = ciclo_guloso(G, inicio, mais_leve=True)
print(f"Ciclo guloso começando e terminando em '{inicio}':", ciclo)

Ciclo completo
Ciclo guloso começando e terminando em 'images-developed-png-8bit/panorama360raw-11.png': ['images-developed-png-8bit/panorama360raw-11.png', 'images-developed-png-8bit/panorama360raw-4.png', 'images-developed-png-8bit/panorama360raw-16.png', 'images-developed-png-8bit/panorama360raw-21.png', 'images-developed-png-8bit/panorama360raw-10.png', 'images-developed-png-8bit/panorama360raw-23.png', 'images-developed-png-8bit/panorama360raw-17.png', 'images-developed-png-8bit/panorama360raw-6.png', 'images-developed-png-8bit/panorama360raw-28.png', 'images-developed-png-8bit/panorama360raw-9.png', 'images-developed-png-8bit/panorama360raw-22.png', 'images-developed-png-8bit/panorama360raw-8.png', 'images-developed-png-8bit/panorama360raw-18.png', 'images-developed-png-8bit/panorama360raw-12.png', 'images-developed-png-8bit/panorama360raw-5.png', 'images-developed-png-8bit/panorama360raw-24.png', 'images-developed-png-8bit/panorama360raw-13.png', 'images-developed-png-8bit/panor